In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os # for handling the directory
import shutil
import json
import tqdm

In [10]:
train_json = '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Datasets/Publaynet/Publaynet_partition_1/Synthetics_DS_partition1/train.json'
with open(train_json) as f:
    train_data = json.load(f)

val_json = '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Datasets/Publaynet/Publaynet_partition_1/Synthetics_DS_partition1/val.json'
with open(val_json) as f:
    val_data = json.load(f)

In [11]:
# Create an auxiliary dictionary to map image_id to filename
def obtain_new_dict(data):
    id_to_filename = {image['id']: {'file_name': image['file_name'], 'width': image['width'], 'height': image['height']} for image in data['images']}

    # Initialize your dictionary with filenames as keys
    new_dict = {values['file_name']: {'width': values['width'], 'height': values['height'], 'images': []} for values in id_to_filename.values()}

    # Iterate through annotations a single time, adding them to the corresponding list in new_dict
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        if image_id in id_to_filename:  # Check if the image_id exists in the mapping
            filename = id_to_filename[image_id]['file_name']
            new_dict[filename]['images'].append(annotation)
    return new_dict


new_dict_train = obtain_new_dict(train_data)
new_dict_val = obtain_new_dict(val_data)

In [13]:
def create_txt_files(data, folder_path):
    for file_name in data.keys():
        with open(f'{folder_path}{file_name[:-4]}.txt', 'w') as f:
            for annotation in data[file_name]['images']:
                x, y, w, h = annotation['bbox']
                x_center = x + w/2
                y_center = y + h/2
                x_center /= data[file_name]['width']
                y_center /= data[file_name]['height']
                w /= data[file_name]['width']
                h /= data[file_name]['height']
                class_id = annotation['category_id'] - 1
                f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

# Create the txt files for the train and val datasets
create_txt_files(new_dict_train, '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Nil/yolo_dir/data_part1/labels/train/')
create_txt_files(new_dict_val, '/hhome/ps2g07/document_analysis/github/Project_Synthesis2-/Nil/yolo_dir/data_part1/labels/val/')

In [14]:
[{'supercategory': '', 'id': 1, 'name': 'text'},
 {'supercategory': '', 'id': 2, 'name': 'title'},
 {'supercategory': '', 'id': 3, 'name': 'list'},
 {'supercategory': '', 'id': 4, 'name': 'table'},
 {'supercategory': '', 'id': 5, 'name': 'figure'},
 {'supercategory': '', 'id': 6, 'name': 'signature'},
 {'supercategory': '', 'id': 7, 'name': 'stamp'},
 {'supercategory': '', 'id': 8, 'name': 'qr'},
 {'supercategory': '', 'id': 9, 'name': 'barcode'}]

[{'supercategory': '', 'id': 1, 'name': 'text'},
 {'supercategory': '', 'id': 2, 'name': 'title'},
 {'supercategory': '', 'id': 3, 'name': 'list'},
 {'supercategory': '', 'id': 4, 'name': 'table'},
 {'supercategory': '', 'id': 5, 'name': 'figure'},
 {'supercategory': '', 'id': 6, 'name': 'signature'},
 {'supercategory': '', 'id': 7, 'name': 'stamp'},
 {'supercategory': '', 'id': 8, 'name': 'qr'},
 {'supercategory': '', 'id': 9, 'name': 'barcode'}]